# Models - Examples 

### 1) Driver Risk

Credit scores and number of jobs in a year can be used to predict drivers' risk status. Given previous data, we learn that credit scores and number of jobs are distributed by group as follows:

- 95% of drivers in the Preferred group have credit scores between 760 and 810. Average number of jobs is 1.
- 95% of drivers in the Standard group have credit scores between 670 and 740. Average number of jobs is 3.
- 95% of drivers in the High-Risk group have credit scores between 580 and 660. Average number of jobs is 4.

Also, we have that 20% of people belong to the Preferred group, 45% belong to the Standard group, and 35% belong to the High-Risk group.

If we select a new policyholder that had 2 jobs in the previous year and a credit score between 730 and 770, what is his/her posterior probability of being a High-Risk driver? (assume conditional independence).


In [2]:
from neoBayesian.models.continuous.normalNormal import *
from neoBayesian.tools.routines import *
from neoBayesian.models.discrete import PoissonDist

In [3]:
# We can use the normal distribution to model credit scores.
# Given the ranges, we need to find distribution parameters:
Pref_p1 = getParamsFromInterval((760, 810), 95)
Stan_p2 = getParamsFromInterval((670, 740), 95)
High_p3 = getParamsFromInterval((580, 660), 95)

# given the parameters we can find the likelihood for each group
ls1 = getLikelihood(Pref_p1, (730, 770))
ls2 = getLikelihood(Stan_p2, (730, 770))
ls3 = getLikelihood(High_p3, (730, 770))

# we can use the Poisson distribution to model number of jobs
lm1 = PoissonDist(1, 2)
lm2 = PoissonDist(3, 2)
lm3 = PoissonDist(4, 2)

In [4]:
# we can multiply likelihoods since we are assuming independence.
# we use this function to display results in tabular form.
tabulateBayesianAlgorithm(
    ('Preferred', 0.20, ls1*lm1),
    ('Standard', 0.45, ls2*lm2),
    ('High-Risk', 0.35, ls3*lm3)
)

Type          Preferred    Standard    High-Risk
----------  -----------  ----------  -----------
Prior         0.2        0.45        0.35
Likelihood    0.022034   0.0180623   5.1619e-09
Joint         0.0044068  0.00812805  1.80666e-09
Posterior     0.351564   0.648436    1.44131e-07

Marginal Probability = sum([joint probabilities])
= 0.0125


### 2) Referendum 

A referendum is about to occur 1 week from now. Marcus and Epictetus are discussing about the probability of a randomly selected person voting YES on this event. They decide to conduct an experiment by asking 30 randomly selected persons in the street if they would vote YES or No. They decide to use the beta-binomial model. Marcus strongly believes that people would mostly vote Yes, so he decides to use 80% and 5% and the mean and variance of the prior beta distribution. Epictetus is totally agnostic so he will use a non-informative prior. After conducting the survey 19 people answer YES. What is the probability that a randomly selected person would vote YES using this data given both priors?

In [5]:
from neoBayesian.models.continuous.betaBinomial import *

In [6]:
MarcusParams = getMeanVarOrAlphaBeta(mean=0.8, variance=0.1)
# Epictetus uses a non-informative prior
EpictetusParams = getMeanVarOrAlphaBeta(0.5, 0.5)

In [7]:
# MARCUS results
getPosterior(30, 19, MarcusParams)

N: 30, k: 19

Posterior:
	alpha-> 19.48 
	beta-> 11.12, 
	expected-> 0.637 
	variance-> 0.007.



(19.48, 11.12, 0.6366013071895424, 0.007320888698547728)

In [8]:
# EPICTETUS results
getPosterior(30, 19, EpictetusParams)

N: 30, k: 19

Posterior:
	alpha-> 18.75 
	beta-> 10.75, 
	expected-> 0.636 
	variance-> 0.008.



(18.75, 10.75, 0.635593220338983, 0.0075939173310853765)

### 3) Ghosts

Albafica invites Kardia to his house and makes the following statement: “There is a high paranormal activity in this house, and 50% of nights you will see a ghost”. Kardia has never seen a ghost and don’t believe what Albafica says; she replies: “You are exaggerating, even if you happened to see 1 ghost 1 night it doesn’t mean that you see ghosts 50% of the time”. Albafica replies: “I literally mean what I say, I see ghosts 50% of the time. You can stay in my house and you will be convinced”. Kardia accepts the challenge and decides to use the beta-binomial model to find the posterior probability of this event. She wants to be really sure that if she happens to see 1 ghost (or a few of them) the following days, this is not just a coincidence. She is skeptical, so she will use 1/30 as the prior mean (she thinks she might be able to observe 1 ghost in 30 days at most) and 0.01 as the variance. How many days would it take for Kardia to change her believes about Albafica’s claim if we assume that, in fact, you can see ghosts in Albafica’s house 50% of all nights? 

In [9]:
from neoBayesian.models.continuous.betaBinomial import *

In [10]:
kardiaParams = getMeanVarOrAlphaBeta(1/30, 0.01)

In [11]:
iterOverK(kardiaParams, 0.5, 0.5*.99, k=1)

  N    k      Alpha       Beta    Expected    Variance
---  ---  ---------  ---------  ----------  ----------
  2    1    1.07407    3.14815    0.254386  0.0363205
  4    2    3.07407    5.14815    0.373874  0.0253835
  6    3    6.07407    8.14815    0.427083  0.0160741
  8    4   10.0741    12.1481     0.453333  0.0106718
 10    5   15.0741    17.1481     0.467816  0.00749391
 12    6   21.0741    23.1481     0.476549  0.00551609
 14    7   28.0741    30.1481     0.482188  0.00421603
 16    8   36.0741    38.1481     0.486028  0.00332089
 18    9   45.0741    47.1481     0.488755  0.00268041
 20   10   55.0741    57.1481     0.490759  0.00220729
 22   11   66.0741    68.1481     0.492274  0.00184837
 24   12   78.0741    80.1481     0.493446  0.00156986
 26   13   91.0741    93.1481     0.494371  0.00134956
 28   14  105.074    107.148      0.495113  0.00117237


It can roughly take 28 days for Kardia to change her believes (assuming she has seen ghosts for 14 nights). Notice that at day 20 the expected value is already 0.49 or 49%.

### 4) A rare disease

A rare disease known as Enricks follows a Poisson distribution with parameter 'u' for the number of cases each year. From this year’s data it is known that the expected value and the standard deviation are 27 and 3 respectively. After doing some research data is gathered from 6 previous years and the number of cases are:

2012: 20  
2013: 18  
2014: 22  
2015: 25  
2016: 26  
2017: 26  

Calculate the posterior expected value and standard deviation using the Gamma-Poisson model.

In [12]:
from neoBayesian.models.continuous.gammaPoisson import *

In [13]:
beta, theta = getMeanVarOrThetaBeta(mean=27, variance=3**2)
observations = [20, 18, 22, 25, 26, 26]

beta, theta, exp, var = getPosterior(beta, theta, observations)
print('Expected Value:', round(exp, 3))
print('Standard Deviation:', round(var**0.5, 3))

Expected Value: 24.222
Standard Deviation: 1.641


### 5) Restaurant revenues

Enrique has bought a restaurant and has been told by the previous owner that annual revenues fluctuate in an interval of roughly 20,000.00 with 90% probability, and he believes revenues are normally distributed. He also mentioned that in 2018 the average monthly profit was $20,000.00 with precision of 1/1000000. Enrique is a bit confused about these rough estimations so he decides to ask the previous bookkeeper for exact numbers. All he can get from the bookkeeper is an excel sheet with annual profits from 2008 to 2015. Profits are:

2008: 180,000.00  
2009: 165,000.00  
2010: 190,500.00  
2011: 210,300.00  
2012: 196,000.00  
2013: 187,000.00  
2014: 220,400.00  
2015: 163,000.00  

Assuming the good faith of the previous business owner and bookkeeper, Enrique decides to use the owner's information as a prior and the actual data as the observations. what can Enrique conclude about the actual mean and standard deviation for the distribution of revenues? Calculate a 99% credible interval.

In [14]:
import neoBayesian.models.continuous.normalNormal as nn
from numpy import var as npv
from tabulate import tabulate

In [15]:
# prior mean (monthly revenues * 12 months)
priorU = 20000 * 12
# variance is the inverse of precision
priorVar = 1000000

# Observations
obsList = [180000, 165000, 190500, 210300, 196000, 187000, 220400, 163000]
# get variance from values stimated by owner
u, obsVar1 = nn.getParamsFromInterval((0, 20000), 90)
# get variance from actual observations to compare
obsVar2 = npv(obsList)

In [16]:
tp1 = nn.getPosterior(obsList, obsVar1, priorU, priorVar, 99)
tp2 = nn.getPosterior(obsList, obsVar2, priorU, priorVar, 99)

In [17]:
headers = ['variance type', 'expected', 'variance', 'confidence interval']
print(tabulate([('estimated by owner',)+(tp1), ('from observations',)+(tp2)], 
               headers=headers,  floatfmt=".2f"))

variance type         expected    variance  confidence interval
------------------  ----------  ----------  ------------------------
estimated by owner   230928.62   822042.58  (228593.05, 233264.191)
from observations    238871.88   977869.18  (236324.546, 241419.218)
